In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-airline-sentiment/Tweets.csv
/kaggle/input/twitter-airline-sentiment/database.sqlite


In [2]:
import pandas as pd
df = pd.read_csv('/kaggle/input/twitter-airline-sentiment/Tweets.csv')
df

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [3]:
data_len = len(df)
train_data = int(data_len*0.8)
data_len

14640

In [4]:
train_tag = df['airline_sentiment'][:train_data]
train_post = df['text'][:train_data]

test_tag = df['airline_sentiment'][train_data:]
test_post = df['text'][train_data:]

In [5]:
import numpy as np
from keras.layers import Activation, Dense, Dropout,Embedding,LSTM
from keras.models import Sequential
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.text import Tokenizer

In [6]:
num_labels = 3       
vocab_size = 1500   
batch_size = 100


tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(train_post) 


x_train = tokenizer.texts_to_matrix(train_post,mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_post,mode ='tfidf')

encoder = LabelBinarizer()  
encoder.fit(train_tag)  

y_train = encoder.transform(train_tag)
y_test = encoder.transform(test_tag) 

In [7]:
embedding_dim = 50
model = Sequential()
model.add(Embedding(input_dim = vocab_size,output_dim=embedding_dim,input_length=15000))
model.add(LSTM(units = 50, return_sequences = True))
model.add(LSTM(units = 10))
model.add(Dropout(0.5))

model.add(Dense(16))
model.add(Dense(3,activation = "softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15000, 50)         75000     
_________________________________________________________________
lstm (LSTM)                  (None, 15000, 50)         20200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                2440      
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                176       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 97,867
Trainable params: 97,867
Non-trainable params: 0
____________________________________________________

In [8]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [9]:
history = model.fit(x_train,y_train,batch_size=32, epochs = 10)

Epoch 1/10
366/366 [==============================] - 459s 1s/step - loss: 0.9583 - accuracy: 0.6032
Epoch 2/10
366/366 [==============================] - 460s 1s/step - loss: 0.9426 - accuracy: 0.6054
Epoch 3/10
366/366 [==============================] - 464s 1s/step - loss: 0.9419 - accuracy: 0.6054
Epoch 4/10
366/366 [==============================] - 459s 1s/step - loss: 0.9416 - accuracy: 0.6054
Epoch 5/10
366/366 [==============================] - 456s 1s/step - loss: 0.9414 - accuracy: 0.6054
Epoch 6/10
366/366 [==============================] - 459s 1s/step - loss: 0.9414 - accuracy: 0.6054
Epoch 7/10
366/366 [==============================] - 460s 1s/step - loss: 0.9413 - accuracy: 0.6054
Epoch 8/10
366/366 [==============================] - 460s 1s/step - loss: 0.9412 - accuracy: 0.6054
Epoch 9/10
366/366 [==============================] - 462s 1s/step - loss: 0.9412 - accuracy: 0.6054
Epoch 10/10
366/366 [==============================] - 463s 1s/step - loss: 0.9412 - accura

In [10]:
# saving the model
import pickle
model.save('sentiLSTM.h5')
with open('tokenizer.pickle','wb') as f:
  pickle.dump(tokenizer,f,protocol = pickle.HIGHEST_PROTOCOL)
# pickle chai serialization ko lagi chahincha

In [11]:
# loading the model
from keras.models import load_model
model = load_model('sentiLSTM.h5')

# tokenizer = Tokenizer()
# aba read mode ma kholne
# model testing ko lagi
with open('tokenizer.pickle','rb') as f:
  tokenizers = pickle.load(f)

In [12]:
encoder.classes_

array(['negative', 'neutral', 'positive'], dtype='<U8')

In [13]:
labels = np.array(['negative', 'neutral', 'positive'])

In [14]:
b = open('news.txt','w')
news = "This is not very good. I do not like it."
b.write(news)
b.close()

In [15]:
b = open('news.txt','r')
print(b.read())
b.close()

This is not very good. I do not like it.


In [16]:
test_files = ['news.txt']

In [17]:
from pathlib import Path
import pandas as pd
x_data = []
for t_f in test_files:
    t_f_data = Path(t_f).read_text()
    x_data.append(t_f_data)
    
x_data_series = pd.Series(x_data)
x_tokenized = tokenizer.texts_to_matrix(x_data_series,mode ='tfidf')

In [18]:
for x_t in x_tokenized:
    prediction = model.predict(np.array([x_t]))
    print(prediction)
    predicted_label = labels[np.argmax(prediction)]
    print(predicted_label)

[[0.61312187 0.21710531 0.1697728 ]]
negative
